In [1]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

# Tune SN_value for SN models

In [2]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm_train']))
    config_list.append('+ue.sn_value\={}'.format(dpp_config['sn_value']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(dpp_config['eval_subsample']))
    config_list.append('training\=electra_base')
    
    return config_list

def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(mc_config['use_spectralnorm_inference']))

    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=True')
    config_list.append('ue.calibrate\=False')
    
    config_list.append('+ue.sn_value\={}'.format(mc_config['sn_value']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    config_list.append('+data.eval_subsample\={}'.format(mc_config['eval_subsample']))
    
    config_list.append('training\=electra_base')
    
    return config_list

In [3]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                try:
                    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    new_task += f' script=run_glue_dpp_hp.py'

                    args = ' '.join(generate_func[0](conf))

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    validation_subsample = conf['validation_subsample']
                    eval_subsample = conf['eval_subsample']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm_train'] == 'True' else 'no_sn'
                    use_spectralnorm_params = '_sn' if conf['use_spectralnorm_train'] == 'True' else ''
                    reg_params = 'cer' if conf['reg_type'] == 'reg-curr' else conf['reg_type']

                    with open(f'../optimal_hyperparameters/electra/{reg_params}{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml', 'r') as f:
                        params = yaml.load(f, Loader=yaml.FullLoader)

                    num_train_epochs = params['num_train_epochs']
                    args += f' training.num_train_epochs\={num_train_epochs}'
                    learning_rate = params['learning_rate']
                    args += f' training.learning_rate\={learning_rate}'
                    per_device_train_batch_size = params['per_device_train_batch_size']
                    args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                    weight_decay = params['weight_decay']
                    args += f' +training.weight_decay\={weight_decay}'

                    try:
                        margin = params['margin']
                        lamb_intra = params['lamb_intra']
                    except:
                        margin = 0
                        lamb_intra = 0

                    args += f' +ue.margin\={margin}'
                    args += f' +ue.lamb_intra\={lamb_intra}'
                    lamb = params['lamb']
                    args += f' ue.lamb\={lamb}'

                    new_task += f" args='{args}'"
                    new_task += f' task_configs={task_configs}'

                    sn_value = conf['sn_value']
                    new_task+=f' output_dir=../workdir/run_train_models_sn_params/electra_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}'

                    full_config += '\n' + new_task if len(full_config) else new_task



                    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
                    task_configs = f'{task_name}.yaml'

                    new_task = copy.deepcopy(base_arg)
                    new_task += f' cuda_devices={cuda_devices}'
                    args = ' '.join(generate_func[1](conf))
                    new_task += f" args='{args}'"
                    new_task += f' config_path=../configs/{task_configs}'
                    new_task += f' script=run_glue_dpp_hp.py'

                    reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                    validation_subsample = conf['validation_subsample']
                    eval_subsample = conf['eval_subsample']
                    use_spectralnorm = 'sn' if conf['use_spectralnorm_train'] == 'True' else 'no_sn'
                    sn_value = conf['sn_value']

                    base_path=f' output_dir=../workdir/run_tasks_for_model_series_sn_params/electra_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}'

                    ue_type = conf['ue']
                    base_path+= f'/{ue_type}'
                    new_task+=base_path

                    model_series_dir=f'../workdir/run_train_models_sn_params/electra_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}/models/{task_name}/'
                    new_task+=f' model_series_dir={model_series_dir}'

                    calc_output_dir = f'../workdir/run_calc_ues_metrics_sn_params/electra_{reg}_{use_spectralnorm}/{task_name}/{sn_value}/{ue_type}'
                    calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series_sn_params/electra_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                    full_config += '\n' + new_task if len(full_config) else new_task
                    full_config += '\n' + calc_command


                    full_config += f'\nrm -rf {model_series_dir}'
                except:
                    pass
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [4]:
tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'calibrate': ['False'],
    'ue': ['mahalanobis'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'validation_subsample': [0.0],
    'eval_subsample': [0.2],
    'sn_value': [0.4, 0.6, 0.8, 1, 2, 3],
    'use_spectralnorm_inference': ['False'],
    'use_spectralnorm_train': ['True'],
}
    
cuda_devices = '[0,1,2,3,4,5]'


generate_bash([ue_configs], cuda_devices, tasks, 
              [generate_train_str_config, generate_ue_eval_methods_str_config], filename='run_train_eval_det_models_with_new_sn_params.sh')

In [5]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm_train']))
    config_list.append('+ue.sn_value\={}'.format(dpp_config['sn_value']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['subsample_perc_val']))
    config_list.append('data.subsample_perc\={}'.format(dpp_config['subsample_perc']))
    config_list.append('ue.use_cache\=False')
    config_list.append('training\=electra_base')
    
    return config_list

def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(mc_config['use_spectralnorm_inference']))

    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=False')
    config_list.append('ue.calibrate\=False')
    
    config_list.append('+ue.sn_value\={}'.format(mc_config['sn_value']))
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['subsample_perc_val']))
    config_list.append('data.subsample_perc\={}'.format(mc_config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    
    return config_list

In [6]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_conll2003_dpp_hp.py'

                args = ' '.join(generate_func[0](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                eval_subsample = conf['subsample_perc_val']
                use_spectralnorm = 'sn' if conf['use_spectralnorm_train'] == 'True' else 'no_sn'
                use_spectralnorm_params = '_sn' if conf['use_spectralnorm_train'] == 'True' else ''
                reg_params = 'cer' if conf['reg_type'] == 'reg-curr' else conf['reg_type']

                with open(f'../optimal_hyperparameters/electra/{reg_params}{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                                
                try:
                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                except:
                    margin = 0
                    lamb_intra = 0
                    
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                sn_value = conf['sn_value']
                new_task+=f' output_dir=../workdir/run_train_models_sn_params/electra_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}'

                full_config += '\n' + new_task if len(full_config) else new_task
                
                
                
                base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[1](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                new_task += f' script=run_conll2003_dpp_hp.py'
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                eval_subsample = conf['subsample_perc_val']
                use_spectralnorm = 'sn' if conf['use_spectralnorm_train'] == 'True' else 'no_sn'
                sn_value = conf['sn_value']
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series_sn_params/electra_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models_sn_params/electra_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}/models/{task_name}/'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics_sn_params/electra_{reg}_{use_spectralnorm}/{task_name}/{sn_value}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series_sn_params/electra_{reg}_{use_spectralnorm}/{task_name}/{eval_subsample}/{sn_value}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
                
                
                full_config += f'\nrm -rf {model_series_dir}'
    with open (f'../scripts/ner_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [7]:
tasks = ['conll2003']

ue_configs = {
    'calibrate': ['False'],
    'ue': ['mahalanobis'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'subsample_perc_val': [0.1],
    'subsample_perc': [0.1],
    'sn_value': [0.4, 0.6, 0.8, 1, 2, 3],
    'use_spectralnorm_inference': ['False'],
    'use_spectralnorm_train': ['True'],
}

cuda_devices = '[0,1,2,3,4,5]'


generate_bash([ue_configs], cuda_devices, tasks, 
              [generate_train_str_config, generate_ue_eval_methods_str_config], filename='run_train_eval_det_models_with_new_sn_params.sh')

# Generate train script for SN models with new HP

In [8]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))

    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('+ue.sn_value\={}'.format(dpp_config['sn_value']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    config_list.append('training\=electra_base')
    
    return config_list

In [9]:
sn_values = {
    'cola': {
        'metric': 0.4,
        'reg': 0.4,
        'raw': 1,
    },
    'mrpc': {
        'metric': 0.4,
        'reg': 3,
        'raw': 1,
    },
    'sst2': {
        'metric': 2,
        'reg': 0.8,
        'raw': 0.8,
    },
    'conll2003': {
        'metric': 3,
        'reg': 1,
        'raw': 3,
    },
}

In [10]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices} seeds=[23419,705525,4837,10671619,1084218,43]'
                new_task += f' script=run_glue.py'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                conf['sn_value'] = sn_values[task_name][reg]
                args = ' '.join(generate_func[i](conf))
                
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''
                reg_params = 'cer' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                                
                with open(f'../optimal_hyperparameters/electra/{reg_params}{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                                
                try:
                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                except:
                    margin = 0
                    lamb_intra = 0
                    
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                new_task+=f' output_dir=../workdir/run_train_models_sn/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'

                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/miscl_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [11]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.0],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_electra_with_sn_params.sh')

# MD SN with new model HP

In [12]:
def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(mc_config['use_spectralnorm']))

    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(mc_config['reg_type']))
    
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=True')
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    config_list.append('training\=electra_base')
    
    return config_list

In [13]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                use_spectralnorm_model = 'sn' if conf['use_spectralnorm_model'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series_sn/electra_{reg}_{use_spectralnorm_model}/{task_name}/{validation_subsample}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models_sn/electra_{reg}_{use_spectralnorm_model}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics_sn/electra_{reg}_{use_spectralnorm_model}/{task_name}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series_sn/electra_{reg}_{use_spectralnorm_model}/{task_name}/{validation_subsample}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../scripts/miscl_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [14]:
tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['mahalanobis'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['False'],
    'use_spectralnorm_model': ['True']
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_det_models_with_sn_params.sh')

In [15]:
full_test = 'bash run_train_models_electra_with_sn_params.sh\n'
full_test += 'bash run_eval_det_models_with_sn_params.sh'

with open (f'../scripts/miscl_scripts/run_electra_new_sn_params.sh', 'w') as rsh:
    rsh.write(full_test)

# New MD SN for CoNLL-2003

In [16]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('+ue.sn_value\={}'.format(dpp_config['sn_value']))

    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(dpp_config['subsample_perc']))
    config_list.append('training\=electra_base')
    
    return config_list

In [17]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices} seeds=[23419,705525,4837,10671619,1084218,43]'
                new_task += f' script=run_conll2003.py'

                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                conf['sn_value'] = sn_values[task_name][reg]
                args = ' '.join(generate_func[i](conf))
                
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''
                reg_params = 'cer' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                                
                with open(f'../optimal_hyperparameters/electra/{reg_params}{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                                
                try:
                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                except:
                    margin = 0
                    lamb_intra = 0
                    
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                new_task+=f' output_dir=../workdir/run_train_models_sn/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'

                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/ner_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [18]:
tasks = ['conll2003']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['True'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_ner_sn.sh')

In [19]:
def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(mc_config['use_spectralnorm']))

    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=True')
    config_list.append('ue.calibrate\=False')
    config_list.append('data.subsample_perc\={}'.format(mc_config['subsample_perc']))
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['validation_subsample']))
    config_list.append('training\=electra_base')
    
    return config_list

In [20]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f' script=run_conll2003.py'
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                use_spectralnorm_model = 'sn' if conf['use_spectralnorm_model'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series_sn/electra_{reg}_{use_spectralnorm_model}/{task_name}/{validation_subsample}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models_sn/electra_{reg}_{use_spectralnorm_model}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics_sn/electra_{reg}_{use_spectralnorm_model}/{task_name}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series_sn/electra_{reg}_{use_spectralnorm_model}/{task_name}/{validation_subsample}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../scripts/ner_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['conll2003']

ue_configs = {
    'ue': ['mahalanobis'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1],
    'use_spectralnorm': ['False'],
    'use_spectralnorm_model': ['True']
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_conll2003_det_models_sn.sh')

In [21]:
full_test = 'bash run_train_models_ner_sn.sh\n'
full_test += 'bash run_eval_conll2003_det_models_sn.sh'

with open (f'../scripts/ner_scripts/run_electra_new_sn_params.sh', 'w') as rsh:
    rsh.write(full_test)

# Generate train script for all models

In [22]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    config_list.append('training\=electra_base')
    
    return config_list

In [23]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices} seeds=[23419,705525,4837,10671619,1084218,43]'
                new_task += f' script=run_glue.py'
                args = ' '.join(generate_func[i](conf))
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''
                reg_params = 'cer' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                                
                with open(f'../optimal_hyperparameters/electra/{reg_params}{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                                
                try:
                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                except:
                    margin = 0
                    lamb_intra = 0
                    
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'

                full_config += '\n' + new_task if len(full_config) else new_task
    with open (f'../scripts/ue_scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [24]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'use_spectralnorm': ['False', 'True'],
    'validation_subsample': [0.0],
    'use_spectralnorm_params': ['False'],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_train_models_electra.sh')

# Generate train script for NER

In [25]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=mc do_ue_estimate\=False')
    
    use_selective = 'False' if dpp_config['reg_type']=='raw' else 'True'
    config_list.append('ue.reg_type\={}'.format(dpp_config['reg_type']))
    config_list.append('ue.use_selective\={}'.format(use_selective))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('+ue.use_spectralnorm\={}'.format(dpp_config['use_spectralnorm']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['validation_subsample']))
    config_list.append('training\=electra_base')
    
    return config_list

In [26]:
def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices} seeds=[23419,705525,4837,10671619,1084218,43]'
                new_task += f' script=run_conll2003.py'
                args = ' '.join(generate_func[i](conf)) + ' data.subsample_perc\=0.1 ue.calibrate\=True '
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                use_spectralnorm_params = '_sn' if conf['use_spectralnorm'] == 'True' else ''
                reg_params = 'cer' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                                
                with open(f'../optimal_hyperparameters/electra/{reg_params}{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml', 'r') as f:
                    params = yaml.load(f, Loader=yaml.FullLoader)

                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'
                                
                try:
                    margin = params['margin']
                    lamb_intra = params['lamb_intra']
                except:
                    margin = 0
                    lamb_intra = 0
                    
                args += f' +ue.margin\={margin}'
                args += f' +ue.lamb_intra\={lamb_intra}'
                lamb = params['lamb']
                args += f' ue.lamb\={lamb}'
        
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                                
                new_task+=f' output_dir=../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'

                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/ue_scripts/run_train_models_ner_full.sh', 'w') as rsh:
        rsh.write(full_config)

In [27]:
tasks = ['conll2003']

train_configs = {
    'calibrate': ['True'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'use_spectralnorm': ['False', 'True'],
    'validation_subsample': [0.1]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

# Generate eval script for DPP and MC-Dropout

In [28]:
import os 

def generate_eval_dpp_str_config(dpp_config):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.max_frac\={}'.format(dpp_config['max_frac']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    
    return config_list


def generate_eval_mc_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    config_list.append('training\=electra_base')

    config_list.append('ue.dropout_subs\={}'.format(mc_config['dropout_subs']))

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    config_list.append('ue.inference_prob\={}'.format(mc_config['inference_prob']))
    
    return config_list

def generate_bash_dpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'
                
                use_ood_sampling = conf['use_ood_sampling']
                if 'max_frac' not in conf.keys():
                    ddpp_type = 'ddpp_ood' if use_ood_sampling=='True' else 'ddpp_dpp'
                    ds_to_mf = {
                        'mrpc': {'ddpp_ood': 0.4, 'ddpp_dpp': 0.55},
                        'cola': {'ddpp_ood': 0.45, 'ddpp_dpp': 0.4},
                        'sst2': {'ddpp_ood': 0.35, 'ddpp_dpp': 0.45}
                    }
                    conf['max_frac'] = ds_to_mf[task_name][ddpp_type]

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_glue.py'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                max_frac = conf['max_frac']
                dpp_type = 'raw' if conf['is_reused_mask'] == 'False' else  'ood' if use_ood_sampling=='True' else 'dpp'
                base_path+= f'/ddpp_{dpp_type}'                
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'
                
                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/ddpp_{dpp_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/ddpp_{dpp_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command

            
    with open (f'../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [29]:
tasks = ['mrpc', 'cola', 'sst2']

dpp_configs = {
    'ue': ['mc-dpp'],
    #'max_frac' : [0.4],
    "committee_size": [50],
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False'],
    'use_ood_sampling': ['False'],
    'dropout_committee_size': [50],
    'validation_subsample': [0.0],
    'is_reused_mask': ['False'],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash_dpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_raw_dpp_models.sh')

In [30]:
tasks = ['mrpc', 'cola', 'sst2']

dpp_configs = {
    'ue': ['mc-dpp'],
    "committee_size": [100],
    'calibrate': ['True'],
    'reg_type': ['raw', 'reg-curr', 'metric'],
    'use_spectralnorm': ['False'],
    'use_ood_sampling': ['False', 'True'],
    'dropout_committee_size': [50],
    'validation_subsample': [0.0],
    'is_reused_mask': ['True'],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash_dpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_ddpp_models.sh')

In [31]:
def generate_bash_mc(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_glue.py'

                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                dropout_subs = conf['dropout_subs']
                base_path+= f'/mc_{dropout_subs}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/mc_all'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/mc_{dropout_subs} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
            
    with open (f'../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)
        
tasks = ['mrpc', 'cola', 'sst2']

mc_configs = {
    'ue': ['mc'],
    'dropout_subs': ['all'],
    "committee_size": [20],
    'calibrate': ['False'],
    'inference_prob': [0.1],
    'reg_type': ['reg-curr', 'raw'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.0],
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash_mc([mc_configs], cuda_devices, tasks, [generate_eval_mc_str_config], file_name='run_eval_mc_models.sh')

# Generate eval script for determenistic methods

In [32]:
def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(mc_config['use_spectralnorm']))

    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=True')
    config_list.append('ue.calibrate\=False')
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    config_list.append('training\=electra_base')
    
    return config_list

In [33]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_glue.py'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['mrpc', 'cola', 'sst2']

ue_configs = {
    'ue': ['mahalanobis'],
    'reg_type': ['reg-curr', 'raw'],
    'validation_subsample': [0.0],
    'use_spectralnorm': ['True', 'False'],
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_det_models.sh')

# Generate scripts for train Deep Ensemble

In [34]:
def generate_bash(cuda_devices, tasks):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_ensemble_series.yaml python ./run_train_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'
        subsample_perc = 0.9 if task_name!='sst2' else 0.1
        args = f'data.subsample_perc\={subsample_perc} ue.calibrate\=True data.validation_subsample\=0.1'      
        
        use_spectralnorm_params = ''
        reg_params = 'raw' 

        with open(f'../optimal_hyperparameters/electra/{reg_params}{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml', 'r') as f:
            params = yaml.load(f, Loader=yaml.FullLoader)


        num_train_epochs = params['num_train_epochs']
        args += f' training.num_train_epochs\={num_train_epochs}'
        learning_rate = params['learning_rate']
        args += f' training.learning_rate\={learning_rate}'
        per_device_train_batch_size = params['per_device_train_batch_size']
        args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
        weight_decay = params['weight_decay']
        args += f' +training.weight_decay\={weight_decay}'       
        
        output_dir = f'../workdir/run_train_ensemble_series/{task_name}/electra/'
        
        new_task = copy.deepcopy(base_arg)
        new_task += f" task_configs={task_configs} args='{args}' output_dir='{output_dir}' cuda_devices={cuda_devices}"
        new_task += f' script=run_glue.py'
        full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/ue_scripts/run_train_ensemble_models.sh', 'w') as rsh:
        rsh.write(full_config)

In [35]:
tasks = ['mrpc', 'cola', 'sst2']

cuda_devices = '[0,1,2,3,4,5]'

generate_bash(cuda_devices, tasks)

# Generate scripts for eval Deep Ensemble

In [36]:
def generate_bash_eval_ens(cuda_devices, tasks, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_ensemble_series.yaml python run_tasks_for_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'

        new_task = copy.deepcopy(base_arg)
        new_task += f' cuda_devices={cuda_devices}'
        new_task += f' config_path=../configs/{task_configs}'

        ensemble_series_dir = f'../workdir/run_train_ensemble_series/{task_name}/electra/ensembles/'
        new_task+=f' ensemble_series_dir={ensemble_series_dir}'
        
        output_dir = f'../workdir/run_tasks_for_ensemble_series/{task_name}/electra/ensembles'
        new_task+=f' output_dir={output_dir}'
        new_task += f' script=run_glue.py'

        calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_raw_no_sn/{task_name}/ensemble'
        calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir={output_dir}/final_results extract_config=False output_dir={calc_output_dir};'

        full_config += '\n' + new_task if len(full_config) else new_task
        full_config += '\n' + calc_command
            
    with open (f'../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [37]:
tasks = ['mrpc', 'cola', 'sst2']

cuda_devices = '0'

generate_bash_eval_ens(cuda_devices, tasks, 'run_eval_ensemble_series.sh')

# NER

## MC Dropout

In [38]:
import os 

def generate_eval_dpp_str_config(dpp_config):
    config_list = []
    config_list.append('ue\={}'.format(dpp_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['validation_subsample']))
    
    config_list.append('training\=electra_base')
        
    config_list.append('ue.dropout.is_reused_mask\={}'.format(dpp_config['is_reused_mask']))
    config_list.append('ue.dropout.max_frac\={}'.format(dpp_config['max_frac']))
    config_list.append('ue.dropout.committee_size\={}'.format(dpp_config['dropout_committee_size']))
    
    config_list.append('ue.committee_size\={}'.format(dpp_config['committee_size']))
    config_list.append('ue.dropout.use_ood_sampling\={}'.format(dpp_config['use_ood_sampling']))
    
    return config_list


def generate_eval_mc_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['validation_subsample']))
    
    config_list.append('training\=electra_base')

    config_list.append('ue.dropout_subs\={}'.format(mc_config['dropout_subs']))

    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    config_list.append('ue.inference_prob\={}'.format(mc_config['inference_prob']))
    
    return config_list

def generate_bash_dpp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'
                
                use_ood_sampling = conf['use_ood_sampling']
                if 'max_frac' not in conf.keys():
                    ddpp_type = 'ddpp_ood' if use_ood_sampling=='True' else 'ddpp_dpp'
                    ds_to_mf = {
                        'conll2003': {'ddpp_ood': 0.6, 'ddpp_dpp': 0.6},
                    }
                    conf['max_frac'] = ds_to_mf[task_name][ddpp_type]

                new_task = copy.deepcopy(base_arg)
                new_task += f' script=run_conll2003.py'
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                max_frac = conf['max_frac']
                dpp_type = 'raw' if conf['is_reused_mask'] == 'False' else  'ood' if use_ood_sampling=='True' else 'dpp'
                base_path+= f'/ddpp_{dpp_type}'                
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'
                
                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/ddpp_{dpp_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/ddpp_{dpp_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command

            
    with open (f'../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

## DDPP

In [39]:
tasks = ['conll2003']

dpp_configs = {
    'ue': ['mc-dpp'],
    "committee_size": [100],
    'calibrate': ['True'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False'],
    'use_ood_sampling': ['False', 'True'],
    'dropout_committee_size': [20],
    'validation_subsample': [0.1],
    'is_reused_mask': ['True'],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash_dpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_conll2003_dpp_models.sh')

In [40]:
tasks = ['conll2003']

dpp_configs = {
    'ue': ['mc-dpp'],
    #'max_frac' : [0.4],
    "committee_size": [20],
    'calibrate': ['False'],
    'reg_type': ['raw'],
    'use_spectralnorm': ['False'],
    'use_ood_sampling': ['False'],
    'dropout_committee_size': [20],
    'validation_subsample': [0.1],
    'is_reused_mask': ['False'],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash_dpp([dpp_configs], cuda_devices, tasks, [generate_eval_dpp_str_config], file_name='run_eval_conll2003_raw_dpp_models.sh')

# MC Dropout

In [41]:
def generate_bash_mc(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f' script=run_conll2003.py'
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                dropout_subs = conf['dropout_subs']
                base_path+= f'/mc_{dropout_subs}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/mc_all'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/mc_{dropout_subs} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
            
    with open (f'../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)
        
tasks = ['conll2003']

mc_configs = {
    'ue': ['mc'],
    'dropout_subs': ['all'],
    "committee_size": [20],
    'calibrate': ['False'],
    'inference_prob': [0.1],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.1],
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash_mc([mc_configs], cuda_devices, tasks, [generate_eval_mc_str_config], file_name='run_eval_conll2003_mc_models.sh')

# Determenistic

In [42]:
def generate_ue_eval_methods_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('ue.use_spectralnorm\={}'.format(mc_config['use_spectralnorm']))

    use_selective = 'False' if mc_config['reg_type']=='raw' else 'True'
    config_list.append('ue.use_selective\={}'.format(use_selective))
    
    config_list.append('do_ue_estimate\=True')
    config_list.append('ue.use_cache\=True')
    config_list.append('ue.calibrate\=False')
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['validation_subsample']))
    config_list.append('training\=electra_base')
    
    return config_list

In [43]:
import os

def generate_bash(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f' script=run_conll2003.py'
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                reg = 'reg' if conf['reg_type'] == 'reg-curr' else conf['reg_type']
                validation_subsample = conf['validation_subsample']
                use_spectralnorm = 'sn' if conf['use_spectralnorm'] == 'True' else 'no_sn'
                
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}'
                
                ue_type = conf['ue']
                base_path+= f'/{ue_type}'
                new_task+=base_path
                
                model_series_dir=f'../workdir/run_train_models/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_{reg}_{use_spectralnorm}/{task_name}/{ue_type}'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series/electra_{reg}_{use_spectralnorm}/{task_name}/{validation_subsample}/{ue_type} extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

tasks = ['conll2003']

ue_configs = {
    'ue': ['mahalanobis'],
    'reg_type': ['metric', 'reg-curr', 'raw'],
    'validation_subsample': [0.1],
    'use_spectralnorm': ['True', 'False'],
}

cuda_devices = '[0,1,2,3,4,5]'

generate_bash([ue_configs], cuda_devices, tasks, [generate_ue_eval_methods_str_config], file_name='run_eval_conll2003_det_models.sh')

## SNGP

### Train

In [44]:
sngp_params = {
    'conll2003': {
        'ridge_factor': 0.0001,
        'momentum': 0.999,
        'num_train_epochs': 85,
        'learning_rate': 3e-5,
        'per_device_train_batch_size': 4,
        'weight_decay': 0.1
    },
    'sst2': {
        'ridge_factor': 0.001,
        'momentum': 0.999,
        'num_train_epochs': 6,
        'learning_rate': 2e-5,
        'per_device_train_batch_size': 8,
        'weight_decay': 0.1
    },
    'mrpc': {
        'ridge_factor': 0.0001,
        'momentum': 0.999,
        'num_train_epochs': 9,
        'learning_rate': 3e-5,
        'per_device_train_batch_size': 16,
        'weight_decay': 0
    },
    'cola': {
        'ridge_factor': 0.001,
        'momentum': 0.999,
        'num_train_epochs': 10,
        'learning_rate': 6e-6,
        'per_device_train_batch_size': 16,
        'weight_decay': 0
    },
}

In [45]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=sngp do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.subsample_perc_val\={}'.format(dpp_config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(dpp_config['subsample_perc']))
    
    config_list.append('training\=electra_base')
    return config_list

In [46]:
def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_conll2003.py'
                subsample_perc = conf['subsample_perc']
                args = ' '.join(generate_func[i](conf))
                
                params = sngp_params[task_name]
                ridge_factor = params['ridge_factor']
                args += f' ue.sngp.ridge_factor\={ridge_factor}'
                momentum = params['momentum']
                args += f' ue.sngp.momentum\={momentum}'
                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'                
                
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models/electra_raw_sngp/{task_name}/{subsample_perc}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/ue_scripts/run_train_models_sngp_ner.sh', 'w') as rsh:
        rsh.write(full_config)

In [47]:
tasks = ['conll2003']

train_configs = {
    'calibrate': ['False'],
    'use_selective': ['False'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

### Eval

In [48]:
import os 

def generate_eval_sngp_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.subsample_perc_val\={}'.format(mc_config['validation_subsample']))
    config_list.append('data.subsample_perc\={}'.format(mc_config['subsample_perc']))
    
    config_list.append('training.per_device_eval_batch_size\={}'.format(mc_config['per_device_eval_batch_size']))
    
    config_list.append('training\=electra_base')
    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    
    return config_list


def generate_bash_sngp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f' script=run_conll2003.py'
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                subsample_perc = conf['subsample_perc']
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_raw_sngp/{task_name}/sngp'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_raw_sngp/{task_name}/{subsample_perc}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_raw_sngp/{task_name}/sngp'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir=../workdir/run_tasks_for_model_series/electra_raw_sngp/{task_name}/sngp extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [49]:
tasks = ['conll2003']

eval_configs = {
    'ue': ['sngp'],
    'calibrate': ['False'],
    'use_selective': ['False'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.1],
    'subsample_perc': [0.1],
    'per_device_eval_batch_size': [64],
    'committee_size': [10]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash_sngp([eval_configs], cuda_devices, tasks, [generate_eval_sngp_str_config], file_name='run_eval_sngp_ner.sh')

# Generate scripts for train Deep Ensemble NER

In [50]:
def generate_bash(cuda_devices, tasks):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_ensemble_series.yaml python ./run_train_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'
        args = 'data.subsample_perc\=0.1 ue.calibrate\=True data.subsample_perc_val\=0.1 ue.use_cache\=True ue.use_selective\=False'
        output_dir = f'../workdir/run_train_ensemble_series/{task_name}/electra'
        
        use_spectralnorm_params = ''
        reg_params = 'raw' 

        with open(f'../optimal_hyperparameters/electra/{reg_params}{use_spectralnorm_params}_{task_name}_optimal_hyperparameters.yaml', 'r') as f:
            params = yaml.load(f, Loader=yaml.FullLoader)

        num_train_epochs = params['num_train_epochs']
        args += f' training.num_train_epochs\={num_train_epochs}'
        learning_rate = params['learning_rate']
        args += f' training.learning_rate\={learning_rate}'
        per_device_train_batch_size = params['per_device_train_batch_size']
        args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
        weight_decay = params['weight_decay']
        args += f' +training.weight_decay\={weight_decay}'    
        
        new_task = copy.deepcopy(base_arg)
        new_task += f' script=run_conll2003.py'
        new_task += f" task_configs={task_configs} args='{args}' output_dir='{output_dir}' cuda_devices={cuda_devices}"
        full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/ue_scripts/run_train_ensemble_models_ner.sh', 'w') as rsh:
        rsh.write(full_config)

In [51]:
tasks = ['conll2003']

cuda_devices = '[0,1,2,3,4,5]'

generate_bash(cuda_devices, tasks)

# Generate scripts for eval Deep Ensemble NER

In [52]:
def generate_bash_eval_ens(cuda_devices, tasks, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_ensemble_series.yaml python run_tasks_for_ensemble_series.py'
    for task_name in tasks:
        task_configs = f'{task_name}.yaml'

        new_task = copy.deepcopy(base_arg)
        new_task += f' cuda_devices={cuda_devices}'
        new_task += f' config_path=../configs/{task_configs}'
        args = f'data.subsample_perc\=0.1 ue.calibrate\=True data.subsample_perc_val\=0.1'

        ensemble_series_dir = f'../workdir/run_train_ensemble_series/{task_name}/electra/ensembles/'
        new_task+=f" args='{args}' ensemble_series_dir={ensemble_series_dir} "
        output_dir = f'../workdir/run_tasks_for_ensemble_series/{task_name}/electra/ensembles'
        new_task+=f' output_dir={output_dir}'
        new_task += f' script=run_conll2003.py'

        calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_raw_no_sn/{task_name}/ensemble'
        calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics_ner.py runs_dir={output_dir}/final_results extract_config=False output_dir={calc_output_dir};'

        full_config += '\n' + new_task if len(full_config) else new_task
        full_config += '\n' + calc_command
            
    with open (f'../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [53]:
import os

tasks = ['conll2003']

cuda_devices = '0'

generate_bash_eval_ens(cuda_devices, tasks, 'run_eval_ensemble_series_ner.sh')

# SNGP GLUE

In [54]:
def generate_train_str_config(dpp_config):
    config_list = []
    config_list.append('ue\=sngp do_ue_estimate\=False')
    
    config_list.append('ue.use_selective\={}'.format(dpp_config['use_selective']))
    config_list.append('ue.calibrate\={}'.format(dpp_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(dpp_config['validation_subsample']))
    
    config_list.append('training\=electra_base')
    return config_list

In [55]:
def generate_bash(configs, cuda_devices, tasks, generate_func):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_train_models.yaml python ./run_train_models.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                new_task += f' script=run_glue.py'
                args = ' '.join(generate_func[i](conf))
                
                params = sngp_params[task_name]
                ridge_factor = params['ridge_factor']
                args += f' ue.sngp.ridge_factor\={ridge_factor}'
                momentum = params['momentum']
                args += f' ue.sngp.momentum\={momentum}'
                num_train_epochs = params['num_train_epochs']
                args += f' training.num_train_epochs\={num_train_epochs}'
                learning_rate = params['learning_rate']
                args += f' training.learning_rate\={learning_rate}'
                per_device_train_batch_size = params['per_device_train_batch_size']
                args += f' training.per_device_train_batch_size\={per_device_train_batch_size}'
                weight_decay = params['weight_decay']
                args += f' +training.weight_decay\={weight_decay}'      
                
                new_task += f" args='{args}'"
                new_task += f' task_configs={task_configs}'
                
                validation_subsample = conf['validation_subsample']
                new_task+=f' output_dir=../workdir/run_train_models/electra_raw_sngp/{task_name}/{validation_subsample}'
                
                full_config += '\n' + new_task if len(full_config) else new_task
    with open ('../scripts/ue_scripts/run_train_models_sngp.sh', 'w') as rsh:
        rsh.write(full_config)

In [56]:
tasks = ['mrpc', 'cola', 'sst2']

train_configs = {
    'calibrate': ['False'],
    'use_selective': ['False'],
    'validation_subsample': [0.0],
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config])

In [57]:
import os 

def generate_eval_sngp_str_config(mc_config):
    config_list = []
    
    config_list.append('ue\={}'.format(mc_config['ue']))
    config_list.append('do_ue_estimate\=True')
    
    config_list.append('ue.calibrate\={}'.format(mc_config['calibrate']))
    config_list.append('data.validation_subsample\={}'.format(mc_config['validation_subsample']))
    
    config_list.append('training.per_device_eval_batch_size\={}'.format(mc_config['per_device_eval_batch_size']))
    
    config_list.append('training\=electra_base')
    config_list.append('ue.committee_size\={}'.format(mc_config['committee_size']))
    
    return config_list


def generate_bash_sngp(configs, cuda_devices, tasks, generate_func, file_name = ''):
    full_config = 'cd ../../src'
    base_arg = 'HYDRA_CONFIG_PATH=../configs/run_tasks_for_model_series.yaml python run_tasks_for_model_series.py'
    for task_name in tasks:
        for i, mc_configs in enumerate(configs):
            for conf in expand_config(mc_configs):
                task_configs = f'{task_name}.yaml'

                new_task = copy.deepcopy(base_arg)
                new_task += f' cuda_devices={cuda_devices}'
                args = ' '.join(generate_func[i](conf))
                new_task += f' script=run_glue.py'
                new_task += f" args='{args}'"
                new_task += f' config_path=../configs/{task_configs}'
                
                validation_subsample = conf['validation_subsample']
                base_path=f' output_dir=../workdir/run_tasks_for_model_series/electra_raw_sngp/{task_name}/sngp'
                new_task+=base_path
                
                model_series_dir = f'../workdir/run_train_models/electra_raw_sngp/{task_name}/{validation_subsample}/models/{task_name}'
                new_task+=f' model_series_dir={model_series_dir}'

                calc_output_dir = f'../workdir/run_calc_ues_metrics/electra_raw_sngp/{task_name}/sngp'
                calc_command = f'HYDRA_CONFIG_PATH=../configs/run_calc_ues_metrics.yaml python ./run_calc_ues_metrics.py runs_dir=../workdir/run_tasks_for_model_series/electra_raw_sngp/{task_name}/sngp extract_config=False output_dir={calc_output_dir};'

                full_config += '\n' + new_task if len(full_config) else new_task
                full_config += '\n' + calc_command
            
    with open (f'../scripts/ue_scripts/{file_name}', 'w') as rsh:
        rsh.write(full_config)

In [58]:
tasks = ['mrpc', 'cola', 'sst2']

eval_configs = {
    'ue': ['sngp'],
    'calibrate': ['False'],
    'use_selective': ['False'],
    'use_spectralnorm': ['False'],
    'validation_subsample': [0.0],
    'per_device_eval_batch_size': [64],
    'committee_size': [10]
}
    
cuda_devices = '[0,1,2,3,4,5]'

generate_bash_sngp([eval_configs], cuda_devices, tasks, [generate_eval_sngp_str_config], file_name='run_eval_sngp.sh')